In [1]:
#data prefix for output data
yearmonth = "202405"

In [2]:
#import necessary packages
import platform
import os
import sys
import time
#for managing the data
import pandas as pd

#install !pip install webdriver-manager
#webdrivers
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import pathlib

import os
from io import StringIO

from tqdm import tqdm

In [3]:
#get current path
curr_path = os.getcwd()
project_tag = curr_path.split("/")[-1]
grouping = curr_path.split("/")[-2]
output_path = "../../../data/{}/{}/{}/".format(grouping,project_tag,yearmonth )

#create data output path if it doesn't exist
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True) 

In [4]:
#load in level 1 data
df = pd.read_csv(output_path + "1_level_table.csv")
print(df.shape)

(4013, 13)


In [5]:
#open web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [6]:
#place holder for final dataframe list
df_list = []

for i,r in tqdm( df.iterrows() ):
    url = r['url_entity_partial']
    url_entity = r['url_entity_full']
    
    #load the main landing page
    driver.get(url_entity)
    
    try:
        #find disclaimer button
        disclamer_btn = driver.find_element(by='name', value="AcceptDisclaimerButton")
    
        #click the disclaimer button to move to the data page
        disclamer_btn.click()
    except:
        pass

    #place holder for extracted data
    data_url_list = []
    data_url_full_list = []
    
    #locate each table rows
    data_rows = driver.find_elements(By.CLASS_NAME, value="hovereffect")

    #if no inspection data skip to the next url
    if len(data_rows)==0:
        continue
    
    for row in data_rows:
        data_url_list.append( row.get_attribute("onclick") )
    
    url_prefix = "https://inspections.healthunit.com"
    for u in data_url_list:
        temp_url = url_prefix+u.split("'")[1]
        data_url_full_list.append(temp_url)
    
    tables = pd.read_html(StringIO((driver.page_source)))
    table  = tables[1]
    table['url_inspection'] = data_url_list
    table['url_inspection_full'] = data_url_full_list
    table['url_entity_full'] = url_entity

    df_list.append(table)
 

4013it [35:59,  1.86it/s]


In [7]:
final_df = pd.concat(df_list)
print(final_df.shape)

(15570, 8)


In [8]:
final_df

,Inspection Date,Inspection Type,Critical Infractions Count,Non Critical Infractions Count,Colour Posting,url_inspection,url_inspection_full,url_entity_full
0,05/06/2024,Required,0.0,2.0,NaN,location.href = '/Portal/Inspection/Details/3e...,https://inspections.healthunit.com/Portal/Insp...,https://inspections.healthunit.com/Portal/Faci...
1,10/30/2023,Required,0.0,0.0,NaN,location.href = '/Portal/Inspection/Details/60...,https://inspections.healthunit.com/Portal/Insp...,https://inspections.healthunit.com/Portal/Faci...
2,05/13/2023,Required,0.0,1.0,NaN,location.href = '/Portal/Inspection/Details/e1...,https://inspections.healthunit.com/Portal/Insp...,https://inspections.healthunit.com/Portal/Faci...
3,10/18/2022,Required,0.0,1.0,NaN,location.href = '/Portal/Inspection/Details/56...,https://inspections.healthunit.com/Portal/Insp...,https://inspections.healthunit.com/Portal/Faci...
4,02/22/2022,Required,0.0,1.0,NaN,location.href = '/Portal/Inspection/Details/88...,https://inspections.healthunit.com/Portal/Insp...,https://inspections.healthunit.com/Portal/Faci...
...,...,...,...,...,...,...,...,...
8,09/27/2022,Required,NaN,NaN,NaN,location.href = '/Portal/Inspection/Details/22...,https://inspections.healthunit.com/Portal/Insp...,https://inspections.healthunit.com/Portal/Faci...
9,04/06/2022,Required,NaN,NaN,NaN,location.href = '/Portal/Inspection/Details/b3...,https://inspections.healthunit.com/Portal/Insp...,https://inspections.healthunit.com/Portal/Faci...
10,03/08/2022,Required,NaN,NaN,NaN,location.href = '/Portal/Inspection/Details/ed...,https://inspections.healthunit.com/Portal/Insp...,https://inspections.healthunit.com/Portal/Faci...
11,12/14/2021,Required,NaN,NaN,NaN,location.href = '/Portal/Inspection/Details/1b...,https://inspections.healthunit.com/Portal/Insp...,https://inspections.healthunit.com/Portal/Faci...


In [9]:
final_df.to_csv( output_path + "2_level_table.csv", index=False)

In [10]:
#close webdriver
try:
    driver.close()
except:
    pass